In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import pandas as pd
data = pd.read_excel("Sample_data.xlsx")
data.head()

,C,n,M
0,159.00,2,259.00
1,129.00,45,174.00
2,115.31,21,154.00
3,111.25,5,111.25
4,111.69,7,111.69


In [3]:
import pulp

samples = data.to_dict('index')

# Define constants
alpha = 200
beta = 0.15

# Create a LP problem
problem = pulp.LpProblem("Maximize Marker Rate", pulp.LpMaximize)

# Define decision variables
R = {sample: pulp.LpVariable("sample{}".format(sample), lowBound=samples[sample]['C'], upBound=samples[sample]['M'], cat='Continuous') for sample in samples}

problem += sum((R[sample] - samples[sample]['C']) * samples[sample]['n'] for sample in samples)

# Add constraints
for sample in samples:
  problem += samples[sample]['M'] >= samples[sample]['C']  # M>=C
  problem += sum(R.values()) <= alpha  # R <= \alpha
  problem += sum([(samples[sample]['M'] - R[sample]) / samples[sample]['M'] for sample in samples]) / len(samples) >= beta  # Maximum average deviation

# Solve the problem
problem.solve()


# Print results
print("Status:", pulp.LpStatus[problem.status])

print("Optimal Allocation:")
for i in samples:
  print("Index = {}, C = {}, M = {}, R = {}".format(i, samples[i]['C'], samples[i]['M'], R[i].varValue))

print("Total Objective Value:", pulp.value(problem.objective))

C:\Users\Esysss\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Status: Infeasible
Optimal Allocation:
Index = 0, C = 159.0, M = 259.0, R = 159.0
Index = 1, C = 129.0, M = 174.0, R = 129.0
Index = 2, C = 115.31, M = 154.0, R = 115.31
Index = 3, C = 111.25, M = 111.25, R = 111.25
Index = 4, C = 111.69, M = 111.69, R = 111.69
Index = 5, C = 112.0, M = 115.0, R = 112.0
Index = 6, C = 180.45, M = 220.0, R = 180.45
Index = 7, C = 151.16, M = 220.0, R = 151.16
Index = 8, C = 134.23, M = 180.0, R = 134.23
Index = 9, C = 129.44, M = 180.0, R = 129.44
Index = 10, C = 180.45, M = 220.0, R = 180.45
Index = 11, C = 151.16, M = 220.0, R = 151.16
Index = 12, C = 134.23, M = 180.0, R = 134.23
Index = 13, C = 129.44, M = 180.0, R = 129.44
Index = 14, C = 166.99, M = 219.99, R = 166.99
Index = 15, C = 166.99, M = 219.99, R = -17388.12
Index = 16, C = 224.0, M = 289.0, R = 224.0
Index = 17, C = 214.0, M = 289.0, R = 214.0
Index = 18, C = 219.0, M = 289.0, R = 219.0
Index = 19, C = 159.0, M = 214.0, R = 159.0
Index = 20, C = 159.0, M = 214.0, R = 159.0
Index = 21, C 

In [4]:
#Check variables
def check(index, samples, r):
  condition1 = samples[index]["M"] >= samples[index]["C"]
  condition2 = samples[index]["C"] <= r <= samples[index]["M"]

  return condition1 and condition2

In [5]:
Results = [R[sample].varValue for sample in samples]
problems = []
for i, result in enumerate(Results):
  if not check(i,samples,result):
    print(i)
    problems.append(i)

15


In [6]:
for i in problems:
  print("Index = {}, C = {}, M = {}, R = {}".format(i, samples[i]['C'], samples[i]['M'], Results[i]))

Index = 15, C = 166.99, M = 219.99, R = -17388.12


In [7]:
def objective(data, r, problems):
    with_problems = 0
    without_problems = 0
    for index, row in data.iterrows():
        with_problems += (r[index] - row["C"]) * row['n']
        if i not in problems:
            without_problems += (r[index] - row["C"]) * row['n']

    return with_problems,without_problems

In [8]:
objective(data,Results,problems)

(-1176192.37, 0)

In [9]:
solver_list = pulp.listSolvers(onlyAvailable=True)
print(solver_list)

['PULP_CBC_CMD']
